In [1]:

# a) Import required libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam

import os


In [2]:
# ==========================================================
# Step 1: Load the Pre-trained CNN Model (VGG16 without top)
# ==========================================================
base_model = VGG16(weights='vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',
                   include_top=False,
                   input_shape=(224, 224, 3))

print("✅ Pre-trained VGG16 model loaded successfully.")

✅ Pre-trained VGG16 model loaded successfully.


In [3]:
# ==========================================================
# Step 2: Freeze lower convolutional layers
# ==========================================================
for layer in base_model.layers:
    layer.trainable = False

print("✅ Base layers frozen.")

# ==========================================================
# Step 3: Add custom classifier (fully connected layers)
# ==========================================================
model = Sequential([
    base_model,
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(4, activation='softmax')  
])


✅ Base layers frozen.


In [4]:
# ==========================================================
# Step 4: Compile the model
# ==========================================================
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
# ==========================================================
# Step 5: Prepare the dataset (images in subfolders)
# ==========================================================
data_dir = "dataset"  # your dataset folder

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 80% training, 20% validation
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

print("✅ Dataset loaded successfully.")


print("Number of classes:", train_generator.num_classes)


Found 153 images belonging to 4 classes.
Found 36 images belonging to 4 classes.
✅ Dataset loaded successfully.
Number of classes: 4


In [6]:
# ==========================================================
# Step 6: Train the classifier layers
# ==========================================================
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)

# ==========================================================
# Step 7: Fine-tune (unfreeze top layers)
# ==========================================================
# Unfreeze last 4 convolutional layers
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Re-compile with smaller learning rate
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("🔄 Fine-tuning top layers...")

fine_tune_history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=5
)



C:\Users\UTKARSH BRAHMANKAR\AppData\Roaming\Python\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 22s 5s/step - accuracy: 0.3411 - loss: 1.4866 - val_accuracy: 0.7222 - val_loss: 0.7982
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 21s 4s/step - accuracy: 0.5564 - loss: 1.0206 - val_accuracy: 0.9167 - val_loss: 0.5238
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 24s 5s/step - accuracy: 0.7416 - loss: 0.7029 - val_accuracy: 0.9167 - val_loss: 0.3551
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 22s 5s/step - accuracy: 0.8419 - loss: 0.4806 - val_accuracy: 0.9722 - val_loss: 0.2197
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 24s 5s/step - accuracy: 0.9113 - loss: 0.3382 - val_accuracy: 0.9722 - val_loss: 0.1500
🔄 Fine-tuning top layers...
Epoch 1/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 30s 6s/step - accuracy: 0.9077 - loss: 0.2766 - val_accuracy: 0.9444 - val_loss: 0.1704
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 27s 6s/step - accuracy: 0.8784 - loss: 0.3453 - val_accuracy: 1.0000 - val_loss: 0.1003
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 27s 5s/step - accuracy: 0.9487 - loss: 0.2205 - val_accuracy: 0.9722 - val_l

In [7]:
# ==========================================================
# Step 8: Evaluate and Save the model
# ==========================================================
loss, acc = model.evaluate(val_generator)
print(f"\n✅ Final Validation Accuracy: {acc*100:.2f}%")

model.save("vgg16_transfer_learning_final.h5")
print("✅ Model saved successfully as vgg16_transfer_learning_final.h5")


2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 527ms/step - accuracy: 1.0000 - loss: 0.0706



✅ Final Validation Accuracy: 100.00%
✅ Model saved successfully as vgg16_transfer_learning_final.h5
